In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [6]:
from _secrets import IEX_CLOUD_API_TOKEN

In [49]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
# data

In [10]:
data['latestPrice']
data['marketCap']

2353881364621

In [11]:
cols = ['Ticker', 'Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = cols)

In [12]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['APPL',
                                                   data['latestPrice'],
                                                   data['marketCap'],
                                                   'N/A'],
                                                   index=cols),
                                        ignore_index=True
                                        )

In [13]:
final_dataframe

,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,APPL,136.57,2353881364621,N/A


In [14]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [18]:
final_dataframe = pd.DataFrame(columns = cols)
for symbol_group in chunks(stocks['Ticker'], 100):
    symbol_list = ','.join(symbol_group)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_list}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_group:
        final_dataframe = final_dataframe.append(
                pd.Series([
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index = cols),
            ignore_index = True
        )
    

In [19]:
final_dataframe

,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,A,143.750,43078832514,N/A
1,AAL,22.831,14565208759,N/A
2,AAP,203.010,13573089922,N/A
3,AAPL,137.588,2359863751830,N/A
4,ABBV,114.300,206296415973,N/A
...,...,...,...,...
500,YUM,121.510,35045418708,N/A
501,ZBH,180.269,37619279260,N/A
502,ZBRA,523.180,27892070335,N/A
503,ZION,56.030,9130705920,N/A


In [34]:
portfolio_size = input("Enter the value of your portfolio:")
try:
    val = float(portfolio_size)
except:
    print("Not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


In [43]:
position_size = float(portfolio_size) / len(final_dataframe.index)
n = len(final_dataframe['Ticker'])
for i in range(n):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])

In [44]:
final_dataframe

,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,A,143.750,43078832514,13
1,AAL,22.831,14565208759,86
2,AAP,203.010,13573089922,9
3,AAPL,137.588,2359863751830,14
4,ABBV,114.300,206296415973,17
...,...,...,...,...
500,YUM,121.510,35045418708,16
501,ZBH,180.269,37619279260,10
502,ZBRA,523.180,27892070335,3
503,ZION,56.030,9130705920,35


In [45]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index=False)

In [46]:
background_color = '#0a0a23'
font_color = '#ffffff'

base = {
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
}
string_format = writer.book.add_format(base)
dollar_format = writer.book.add_format({"num_format": '$0.00',}.update(base))
integer_format = writer.book.add_format({'num_format': '0'}.update(base))

In [47]:
cols_F = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}
for col in cols_F:
    writer.sheets['Recommended Trades'].set_column(f'{col}:{col}', 20, cols_F[col][1])
    writer.sheets['Recommended Trades'].write(f'{col}1', cols_F[col][0], string_format)

In [48]:
writer.save()